<a href="https://colab.research.google.com/github/velish-qubadov/Machine-Learning/blob/main/Ensemble_Learning_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Hard Voting

Hard Voting-də hər model müstəqil olaraq bir sinif proqnozlaşdırır və sonda ən çox səs toplayan sinif final nəticə kimi qəbul edilir.

İş prinsipi:

Bir neçə fərqli classifier istifadə olunur (məsələn: Logistic Regression, Random Forest, SVM).

Hər model öz proqnozunu verir.

Hansı sinif ən çox seçilibsə, final nəticə o sinif olu

In [ ]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X,y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train, X_test, y_train, y_test= train_test_split(X,y,random_state=42)
voting_clf= VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc',SVC(random_state=42))
    ]       #skable
)
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [ ]:
voting_clf.named_estimators_

{'lr': LogisticRegression(random_state=42),
 'rf': RandomForestClassifier(random_state=42),
 'svc': SVC(random_state=42)}

In [ ]:
for name, clf in voting_clf.named_estimators_.items():
  print(name, '=', clf.score(X_test,y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [ ]:




voting_clf.predict(X_test[:1])

array([1])

In [ ]:
voting_clf.estimators_

[LogisticRegression(random_state=42),
 RandomForestClassifier(random_state=42),
 SVC(random_state=42)]

In [ ]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]


[array([1]), array([1]), array([0])]

In [ ]:
voting_clf.score(X_test,y_test)

0.912

# Soft Voting

Soft Voting-də hər modelin proqnozlaşdırdığı sinif ehtimalları nəzərə alınır və ortalama hesablanaraq ən yüksək ehtimala sahib sinif seçilir.

İş prinsipi:

Hər model müəyyən sinfə aid olma ehtimalını hesablayır.

Ehtimalların ortalaması götürülür.

Ən yüksək ehtimala malik sinif final nəticə olur.

ehtimallari topluyub sayina bolur ve tresholda gore True False verir, soft>>hard

In [ ]:
voting_clf.voting='soft'
voting_clf.named_estimators['svc'].probability= True
voting_clf.fit(X_train, y_train)

voting_clf.score(X_test,y_test)

0.92

# Bagging and Pasting

Pasting, Bagging ilə oxşardır, amma nümunələri seçərkən təkrar istifadə edilmir (without replacement).


Yəni, hər alt dataset əsas datasetdən unikal nümunələr alır, eyni nümunə iki dəfə istifadə olunmur.

🔹 İş prinsipi:

Əsas datasetdən **təkrar edilmədən** (without replacement) bir neçə alt dataset yaradılır.

Hər model bu fərqli datasetlərdə öyrədilir.

Nəticələr eyni Bagging-də olduğu kimi birləşdirilir (ortalama və ya səs çoxluğu).

Bagging, modelin fərqli təlim dəstlərində öyrədilməsini və nəticələrin ortalamasını (regression üçün) və ya səs çoxluğu prinsipinə (classification üçün) əsaslanaraq final qərar qəbul etməsini təmin edir.

🔹 İş prinsipi:

Əsas məlumat dəstindən (dataset) **təkrar seçmə** ilə (with replacement) bir neçə alt dataset (subsample) yaradılır.

Hər bir alt dataset üçün ayrı-ayrı eyni model öyrədilir (məsələn, bir neçə Decision Tree).

Nəticələr birləşdirilir:

Regression üçün: Ortalama alınır.

Classification üçün: Ən çox seçilən sinif seçilir (Hard Voting)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(DecisionTreeClassifier(),
                          n_estimators=500,
                          max_samples=100,
                          n_jobs=-1,
                          random_state=42
)

bag_clf.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

In [ ]:
bag_clf.score(X_test,y_test)

0.904

# Out-of-Bag Evaluation

Bagging metodunda hər bir baza model (məsələn, Decision Tree) əsas datasetdən təkrarla (with replacement) seçilmiş nümunələrlə öyrədilir. Lakin bu seçmə zamanı bəzi nümunələr seçilmədən qala bilər. Bu seçilməyən nümunələrə Out-of-Bag nümunələr (OOB samples) deyilir.

In [ ]:
bag_clf=BaggingClassifier(DecisionTreeClassifier(),
                          n_estimators=500,
                          oob_score=True,
                          n_jobs=-1,
                          random_state=42
)

bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.896

In [ ]:
from sklearn.metrics import accuracy_score
y_pred= bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.92

# Random Forests

Random Forest, Bagging metodunun xüsusi bir tətbiqidir və çoxsaylı Decision Tree (Qərar Ağacları) modellərinin birləşməsindən ibarətdir. Əsas məqsəd overfitting-i azaltmaq, modelin stabil olmasını təmin etmək və dəqiqliyi artırmaqdır.


İş prinsipi

Bootstrap seçmə (Bagging) – Datasetdən təkrar seçmə (with replacement) ilə müxtəlif alt datasetlər yaradılır.

Fərqli Decision Tree modelləri öyrədilir – Hər ağac yalnız ona verilən alt dataset üzərində qurulur.

Xüsusiyyətlərin təsadüfi seçilməsi – Hər ağac üçün yalnız təsadüfi seçilmiş bəzi xüsusiyyətlər (features) istifadə edilir.

Nəticələrin birləşdirilməsi –

Regression üçün: Ağacların nəticələrinin ortalaması alınır.

Classification üçün: Ən çox səs alan sinif seçilir (Hard Voting).

💡 Əsas fərq: Bagging metodundan fərqli olaraq, Random Forest hər ağacı qurarkən təsadüfi xüsusiyyətlər də seçir. Bu, ağacların daha çox müxtəliflik göstərməsinə və overfitting-in azalmasına səbəb olur.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf= RandomForestClassifier(n_estimators= 500,
                                max_leaf_nodes=16,
                                n_jobs=-1,
                                random_state=42
)
rnd_clf.fit(X_train,y_train)
y_pred_rf=rnd_clf.predict(X_test)

In [ ]:
bag_clf=BaggingClassifier(DecisionTreeClassifier(max_features='sqrt', max_leaf_nodes=16),
                          n_estimators=500,
                          n_jobs=-1,
                          random_state=42
)

In [ ]:
bag_clf.fit(X_train,y_train)
y_pred_bag= bag_clf.predict(X_test)
np.all(y_pred_bag==y_pred_rf)

True

# Feature Importance

Feature Importance (Xüsusiyyətlərin Əhəmiyyəti), bir modelin hansı xüsusiyyətlərə (features) daha çox diqqət yetirdiyini göstərir. Yəni, modelin proqnozları üçün hansı xüsusiyyətlərin daha vacib olduğunu öyrənməyə kömək edir.

📌 Əsas məqsədlər:

✔ Modelin açıq izah edilə bilməsi (interpretability).

✔ Gərəksiz xüsusiyyətləri çıxarmaq və daha effektiv model qurmaq.

✔ Modelin hansı dəyişənlərə əsaslandığını anlamaq.

In [ ]:
from sklearn.datasets import load_iris

iris= load_iris(as_frame=True)
rnd_clf= RandomForestClassifier(n_estimators=500,
                                random_state=42)
rnd_clf.fit(iris.data, iris.target)
for score,name in zip(rnd_clf.feature_importances_, iris.data.columns):
  print(round(score,2)*100, '%', name)

11.0 % sepal length (cm)
2.0 % sepal width (cm)
44.0 % petal length (cm)
42.0 % petal width (cm)


# Boosting

Boosting, zəif modelləri (weak learners) bir-birinin ardınca öyrədərək güclü bir model yaratmağa çalışan ansambl metodudur.

Əsas məqsəd hər yeni modelin əvvəlki modelin səhvlərini düzəltməsidir.

Əksər hallarda zəif model olaraq Decision Tree (ən çox "stump" - yəni dərinliyi 1 olan ağaclar) istifadə olunur.

Bagging-dən fərqi: Bagging paralel işləyir, Boosting isə ardıcıl şəkildə modelləri qurur.

## AdaBoosting

AdaBoost, ilk Boosting metodlarından biridir və hər yeni ağac əvvəlki modelin səhvlərini düzəltməyə çalışır.



In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf= AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=30,
    learning_rate=0.5,
    random_state=42
)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

In [ ]:
ada_clf.score(X_test,y_test)

0.88

## Gradient Boosting

Gradient Boosting, AdaBoost-dan fərqli olaraq səhvləri çəkilərlə deyil, gradient enmə metodu ilə düzəldir.

Əvvəlcə sadə bir model öyrədilir.

Daha sonra modelin **qalıq səhvlərini** proqnozlaşdıran yeni bir model öyrədilir.

Bu proses gradient descent ilə səhvləri azaldana qədər davam edir.

✅ Gradient Boosting (GBM, XGBoost, LightGBM, CatBoost)



In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
np.random.seed(42)

X=np.random.rand(100,1)-0.5
y=3*X[:,0]**2 + 0.05* np.random.randn(100)

tree_reg1=DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [ ]:
y2= y- tree_reg1.predict(X)
tree_reg2=DecisionTreeRegressor(max_depth=2, random_state=43)
tree_reg2.fit(X,y2)


DecisionTreeRegressor(max_depth=2, random_state=43)

In [ ]:
y3= y2- tree_reg2.predict(X)
tree_reg3=DecisionTreeRegressor(max_depth=2, random_state=44)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2, random_state=44)

In [ ]:
X_new= np.array([[-0.4],[0.],[0.5]])
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([0.49484029, 0.04021166, 0.75026781])

In [ ]:
from sklearn.ensemble import  GradientBoostingRegressor
gbrt= GradientBoostingRegressor(max_depth=2,
                                 n_estimators=3,
                                 learning_rate=1.0,
                                 random_state=42)
gbrt.fit(X,y)


GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

Learning rate (
𝜂
η), hər yeni modelin (yəni yeni Decision Tree-nin) əsas modelə nə qədər töhfə verməsini idarə edən hiperparametrdir.

✅ Kiçik learning rate → Daha yavaş öyrənmə, amma daha sabit və dəqiq model.

✅ Böyük learning rate → Daha sürətli öyrənmə, amma overfitting riski var.

In [ ]:
gbrt_best= GradientBoostingRegressor(max_depth=2,
                                 n_estimators=500,
                                 learning_rate=0.05,
                                n_iter_no_change=10,
                                 random_state=42)
gbrt_best.fit(X,y)


GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

In [ ]:
gbrt_best.n_estimators_


92

# StackingClassifier

Stacking (Stacked Generalization), müxtəlif modellərin çıxışlarını birləşdirərək daha yaxşı performans əldə etməyə çalışan ansambl metodudur.

✅ Bagging və Boosting-dən fərqli olaraq, müxtəlif model növlərini birləşdirir.

✅ Ensemble learning-in bir növüdür və fərqli modellərin kombinasiyasından istifadə edir.

✅ İki əsas səviyyədən ibarətdir:

Base (Level-0) Model-lər: Fərqli Machine Learning modelləri (məsələn, Decision Tree, SVM, Logistic Regression).

Meta (Level-1) Model: Base model-lərin proqnozlarını istifadə edərək yekun qərarı verən model (adətən Linear Regression və ya Logistic Regression)

In [ ]:
from sklearn.ensemble import StackingClassifier
stacking_clf= StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc',SVC(probability=True,random_state=42))  # semi-scaleable
    ],
    final_estimator= RandomForestClassifier(random_state=43),
    cv=5
)

stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [ ]:
stacking_clf.score(X_test, y_test)

0.928